# 📘 Bedrock Multi-Model RAG App (Notebook Version)
This notebook lets you:
- Upload **multiple PDFs**
- Create a **FAISS vectorstore**
- Ask a query
- Get answers from **multiple Amazon Bedrock models simultaneously**
- Measure response time for each model


!pip install langchain faiss-cpu boto3 pypdf


In [6]:
%pip install langchain faiss-cpu boto3 pypdf ipywidgets langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.2 MB/s eta 0:00:00a 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import time

In [8]:
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

upload_dir = Path("uploaded_txts")
upload_dir.mkdir(exist_ok=True)

uploader = widgets.FileUpload(accept='.txt', multiple=True)
display(uploader)


FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [9]:
for fileinfo in uploader.value:
    filename = fileinfo['name']
    content = fileinfo['content']
    with open(upload_dir / filename, "wb") as f:
        f.write(content)

print("Uploaded files:", list(upload_dir.glob("*.txt")))


Uploaded files: [PosixPath('uploaded_txts/Guardadito Kids.txt'), PosixPath('uploaded_txts/Social Programs Manual.txt')]


In [10]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_txts(txt_paths, chunk_size=500, chunk_overlap=100):
    all_docs = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    for path in txt_paths:
        loader = TextLoader(str(path), encoding='utf-8')
        docs = loader.load()
        chunks = splitter.split_documents(docs)
        all_docs.extend(chunks)
    return all_docs

documents = load_and_split_txts(upload_dir.glob("*.txt"))
print(f"Loaded {len(documents)} chunks from .txt files.")


Loaded 87 chunks from .txt files.


In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings 

embeddings = BedrockEmbeddings()

vectorstore = FAISS.from_documents(documents, embeddings)
print("✅ FAISS index creado con", len(documents), "fragmentos.")


/var/folders/73/m50h3qv91_n45vnh3sfw1pb00000gq/T/ipykernel_32930/2616746936.py:4: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  embeddings = BedrockEmbeddings()


✅ FAISS index creado con 87 fragmentos.


In [12]:
from langchain.prompts import PromptTemplate

template = """
<instrucciones_agente>
  <proposito>
    Eres un asistente de información bancaria.
    Tu única función es responder exclusivamente con base en los documentos internos proporcionados.
  </proposito>

  <funcion_principal>
    <categorias>
      <categoria>Cuentas de Débito y Tarjetas</categoria>
      <categoria>Ahorro (Guardadito)</categoria>
      <categoria>Inversiones</categoria>
      <categoria>Créditos y Financiamiento</categoria>
      <categoria>Nómina y Portabilidad</categoria>
      <categoria>Pagos y Transferencias</categoria>
      <categoria>Retiros y Efectivo</categoria>
      <categoria>Seguridad y Acceso</categoria>
      <categoria>Servicios Adicionales</categoria>
    </categorias>
  </funcion_principal>

  <protocolo_respuesta>
    <regla>Brindar respuestas breves y precisas</regla>
    <regla>Usar lenguaje natural y amigable</regla>
    <regla>Seguir las políticas internas de SOLTIVA</regla>
    <regla>Mantener las respuestas claras, útiles y concisas</regla>
    <regla>No inventar información ni dar detalles que no estén en los documentos proporcionados</regla>
  </protocolo_respuesta>

  <consultas_fuera_alcance>
    <accion>Rechazar amablemente preguntas fuera de las categorías permitidas</accion>
    <accion>Redirigir al usuario mencionando los temas disponibles</accion>
    <accion>No proporcionar información sobre temas no autorizados</accion>
  </consultas_fuera_alcance>

  <manejo_datos>
    <regla>Utilizar únicamente documentación interna autorizada</regla>
    <regla>No acceder ni compartir datos sensibles de clientes</regla>
    <regla>Cumplir con todos los protocolos de seguridad de la información</regla>
  </manejo_datos>

  <estilo_comunicacion>
    <tono>Profesional y cortés</tono>
    <tono>Consistente con la voz de la marca</tono>
    <tono>Enfocado en la claridad y eficiencia</tono>
    <tono>Respuestas relevantes, fieles al contexto y sin redundancia</tono>
  </estilo_comunicacio


  <reglas>
    <regla>No inventes ni supongas información, aunque parezca razonable.</regla>
    <regla>No uses frases introductorias como "para abrir una cuenta necesitas...". Sé directo.</regla>
    <regla>Si la información no está en los documentos, responde "No tengo esa información."</regla>
    <regla>Tu respuesta no debe superar {word_limit} palabras bajo ninguna circunstancia.</regla>
  </reglas>

</instrucciones_agente>

Contesta solo con base en el siguiente resumen de políticas oficiales internas:

<documentos_contexto>
{context}
</documentos_contexto>

<pregunta_usuario>
{question}
</pregunta_usuario>

<instruccion_final>
Tu respuesta debe tener como máximo {word_limit} palabras. 
Si no hay información suficiente en el contexto, responde claramente "No tengo esa información".
</instruccion_final>
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question", "word_limit"]
)

print("✅ Prompt template con políticas e instrucciones cargado.")

✅ Prompt template con políticas e instrucciones cargado.


In [13]:
import asyncio
import time
import boto3
from botocore.exceptions import ClientError
from IPython.display import Markdown, display
import nest_asyncio
nest_asyncio.apply()

model_ids = [
    # "anthropic.claude-v2",

    # "us.amazon.nova-micro-v1:0",
    # "us.amazon.nova-lite-v1:0",

    "us.anthropic.claude-3-haiku-20240307-v1:0",    
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    # "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    # "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
]

client = boto3.client('bedrock-runtime', region_name='us-east-1')

def invoke_converse(model_id, messages, inference_config):
    return client.converse(
        modelId=model_id,
        messages=messages,
        inferenceConfig=inference_config
    )

async def run_chain(model_id, query, context, word_limit):
    try:
        messages = [
            {
                'role': 'user',
                'content': [{'text': f"{context}\n\nPregunta: {query}\n\nLímite de palabras: {word_limit}"}]
            }
        ]

        inference_config = {
            'maxTokens': 300,
            'temperature': 0.2,
            'topP': 0.9
        }

        start = time.time()

        response = await asyncio.to_thread(invoke_converse, model_id, messages, inference_config)

        output_message = response['output']['message']
        result = output_message['content'][0]['text']

        # Obtener el número de tokens de salida
        output_tokens = response.get('usage', {}).get('outputTokens', 'N/A')

        # Calcular el número aproximado de palabras
        if isinstance(output_tokens, int):
            words_estimate = output_tokens / 2  # Relación tokens-palabras para español
        else:
            words_estimate = 'N/A'

        elapsed = time.time() - start
        return {
            "model": model_id,
            "response": result.strip(),
            "time": round(elapsed, 2),
            "tokens": output_tokens,
            "words_estimate": words_estimate
        }

    except ClientError as e:
        return {
            "model": model_id,
            "response": f"❌ Error: {e.response['Error']['Message']}",
            "time": None,
            "tokens": None,
            "words_estimate": None
        }
    except Exception as e:
        return {
            "model": model_id,
            "response": f"❌ Error: {str(e)}",
            "time": None,
            "tokens": None,
            "words_estimate": None
        }

async def run_all_models(query, docs, word_limit=50):
    context = "\n\n".join([d.page_content for d in docs])
    tasks = [run_chain(mid, query, context, word_limit) for mid in model_ids]
    return await asyncio.gather(*tasks)


In [14]:
query = "cuanto cuesta personalizar el plástico de guardadito kids?"

In [15]:
relevant_docs = vectorstore.similarity_search(query, k=7)

results = await run_all_models(query, relevant_docs, word_limit=50)

for r in results:
    display(Markdown(
        f"### 🧠 {r['model']}\n"
        f"⏱️ {r['time']}s\n"
        f"🔢 Tokens: {r.get('tokens', 'N/A')}\n"
        f"📝 Palabras estimadas: {r.get('words_estimate', 'N/A')}\n\n"
        f"**Respuesta:** {r['response']}"
    ))

### 🧠 us.anthropic.claude-3-haiku-20240307-v1:0
⏱️ 1.19s
🔢 Tokens: 39
📝 Palabras estimadas: 19.5

**Respuesta:** Según la información proporcionada, el costo por personalizar el plástico de la cuenta Guardadito Kids es de $99.14 por evento.

### 🧠 us.anthropic.claude-3-5-haiku-20241022-v1:0
⏱️ 1.42s
🔢 Tokens: 38
📝 Palabras estimadas: 19.0

**Respuesta:** Según la tabla de comisiones, la personalización de plástico para Guardadito Kids tiene un costo de $99.14 por evento.